In [1]:
from model.model import *
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pickle
from dataset import ShapeNetDataset
import numpy as np
import math
%matplotlib inline
from IPython.display import clear_output
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:

def sample_data(dataset,data_type,batch_size):
    pcds,labels,fns = dataset.get_data(data_type,batch_size)
    return pcds,labels
    
def get_all_data(dataset,data_type):
    pcds,labels,fns = dataset.get_data(data_type,batch_size)
    return pcds,labels

def compute_mIoU(preds,target):
    preds = preds.cpu().detach().numpy()
    target = target.cpu().detach().numpy()
    N_of_parts = np.max(target)
    N_of_pts = np.shape(target)[0]
    total_IoU = 0.
    for i in range(N_of_parts):
        U = [False]*N_of_pts
        I = [False]*N_of_pts
        for j in range(N_of_pts):
            if target[j] == i and preds[j] == i:
                I[j] = True
            if target[j] == i or preds[j] == i:
                U[j] = True
        if sum(U) == 0: 
            total_IoU += 1
        else:
            total_IoU += sum(I)/sum(U)
    return total_IoU/N_of_parts


In [3]:
##Hyperparameters
N_CLASSES = 16
EPOCHS = 1000#2000
BATCH_SIZE = 32
INIT_LR = 0.001
MOMENTUM = 0.9
LR_STEP = 20
SCHEDULER_GAMMA = 0.5
EVAL_EVERY = 1
REG_WEIGHT = 0.001
criterion = nn.CrossEntropyLoss()

In [4]:
class_ID = 0
ShapeNetData = ShapeNetDataset('datasets/ShapeNet/',class_ID)
N_of_parts = ShapeNetData.get_N_parts(class_ID)
net = PointNetDenseClassification(N_of_parts).to(device)
optimizer = optim.Adam(params=net.parameters(), lr=INIT_LR)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=LR_STEP, gamma=SCHEDULER_GAMMA)


In [5]:
##training loop
train_losses = []
train_iterations = []
eval_losses = []
eval_iterations = []


for epoch in range(10):
    optimizer.zero_grad()
    net.eval()#to allow for batch of 1
    #load the batch of data
    all_data,all_labels = sample_data(ShapeNetData,'train',BATCH_SIZE)
    mIoU_batch = 0.
    for data,labels in zip(all_data,all_labels):
        data = torch.from_numpy(np.expand_dims(np.array(data),0)).float().to(device)
        labels = torch.from_numpy(np.expand_dims(np.array(labels),0)).to(device)
        
        #compute the loss
        preds,M2 = net(data)
        loss = criterion(preds[0,:,:],labels[0,:])
    
        #add transformation matrix regularization loss
        I = torch.eye(64).unsqueeze(0).to(device)
        loss2 = torch.mean(torch.norm(torch.bmm(M2,M2.transpose(2,1)) - I, dim=(1,2)))    
        loss += REG_WEIGHT*loss2
        
        mIoU_one_part = compute_mIoU(torch.max(preds[0,:,:],dim = 1).values,labels[0,:])
        mIoU_batch +=mIoU_one_part 
        
    train_losses.append(loss.detach().cpu())
    train_iterations.append(epoch)
    print("mIoU:",mIoU_batch/BATCH_SIZE)
    #step the optimizer
    
    loss.backward()
    optimizer.step()
    scheduler.step()
    
    print(loss)
#     if epoch%EVAL_EVERY == 0:
#         with torch.no_grad():
#             net.eval()
#             #load the batch of eval data (batch size couldn't be too big)
#             data,labels = sample_data(data,'eval',32)
#             data = torch.from_numpy(data).float().to(device)
#             labels = torch.from_numpy(labels).to(device)
            
#             preds,M2 = net(data)
#             test_loss = criterion(preds,labels)
#             #add transformation matrix regularization loss
#             I = torch.eye(64).unsqueeze(0).to(device)
#             test_loss2 = torch.mean(torch.norm(torch.bmm(M2,M2.transpose(2,1)) - I, dim=(1,2)))    
#             test_loss += REG_WEIGHT*test_loss2

#             test_losses.append(test_loss.detach().cpu())
#             test_iterations.append(epoch)
# #             test_accuracy = compute_mIoU(preds,labels)
# #             print('Epoch:',epoch, ';train and test accuracies:',train_accuracy,test_accuracy)

# #     clear_output()
#     plt.plot(train_iterations, train_losses, 'b',test_iterations, test_losses, 'r')
#     plt.xlabel('Epoch')
#     plt.ylabel('Loss') 
#     plt.legend(['Train','Test'])
#     plt.title('Epoch vs Loss')
# #     plt.show()
#     plt.savefig("./cls_losses.png") # save graph for training visualization

mIoU: 0.0
tensor(1.4026, device='cuda:0', grad_fn=<AddBackward0>)
mIoU: 0.0
tensor(1.3937, device='cuda:0', grad_fn=<AddBackward0>)
mIoU: 0.0
tensor(1.3891, device='cuda:0', grad_fn=<AddBackward0>)
mIoU: 0.15533037180059814
tensor(1.3886, device='cuda:0', grad_fn=<AddBackward0>)
mIoU: 0.1711923886380641
tensor(1.3882, device='cuda:0', grad_fn=<AddBackward0>)
mIoU: 0.15424808922468053
tensor(1.3879, device='cuda:0', grad_fn=<AddBackward0>)


ZeroDivisionError: division by zero